In [1]:
import pickle
import os
import sys
sys.path.insert(0, '../olympus')
sys.path.insert(0, '../graphnet/src')

import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
import seaborn as sns
import random
from jax import random
from jax import numpy as jnp

import torch
import torch_cluster
import torch_geometric
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from nemesis.event_generation.detector import make_line, generate_noise, Detector, make_triang
from nemesis.plotting import plot_event, plot_events, plot_confusion
from nemesis.data_handling.utils import event_labelling
from nemesis.evaluation import energy_division_loaders, energy_evaluation
from nemesis.node_features import generate_features

from torch.nn import Linear, Identity, ReLU, Softmax, Dropout, LeakyReLU
import torch.nn.functional as F
from torch_geometric.nn import knn_graph, TAGConv, global_mean_pool, global_max_pool, BatchNorm,  global_add_pool, EdgeConv, DynamicEdgeConv

from graphnet.models.gnn.dynedge import DynEdge
from graphnet.components.layers import DynEdgeConv
from graphnet.models.gnn.gnn import GNN
from graphnet.models.utils import calculate_xyzt_homophily, calculate_xyz_homophily_POne

%matplotlib inline


ModuleNotFoundError: No module named 'plotly'

In [ ]:
outpath = "."
plt.rcParams['figure.dpi'] = 100
if torch.cuda.is_available():
    print('CUDA')
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print('CPU')
    
rng = np.random.RandomState(31338)
oms_per_line = 20
dist_z = 50 # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100 # m
pmts_per_module = 16
pmt_cath_area_r = 75E-3 / 2 # m
module_radius = 0.21 # m

efficiency = pmts_per_module * (pmt_cath_area_r)**2 * np.pi / (4*np.pi*module_radius**2)
det = make_triang(side_len, oms_per_line, dist_z, dark_noise_rate, rng, efficiency=efficiency)
module_positions = jnp.asarray(det.module_coords)

In [ ]:
cascades, cascade_records = pickle.load(open(os.path.join(outpath, f"/dss/pone/pone_events/cascades_backup/cascades_1500_0.pickle"), "rb"))
tracks, track_records = pickle.load(open(os.path.join(outpath, f"/dss/pone/pone_events/tracks_backup/tracks_1500_0.pickle"), "rb"))
stracks, strack_records = pickle.load(open(os.path.join(outpath, f"/dss/pone/pone_events/stracks_backup/stracks_1500_0.pickle"), "rb"))

In [ ]:
cascades1, cascade_records1 = pickle.load(open(f"/dss/pone/pone_events/cascades_15000ev_3.5-6.5.pickle", "rb"))
print('cascades DONE!')
tracks1, track_records1 = pickle.load(open(f"/dss/pone/pone_events/tracks_15000ev_3.5-6.5.pickle", "rb"))
print('tracks DONE!')
stracks1, strack_records1 = pickle.load(open(f"/dss/pone/pone_events/stracks_15000ev_3.5-6.5.pickle", "rb"))
print('starting tracks DONE!')
cascades2, cascade_records2 = pickle.load(open(f"/dss/pone/pone_events/cascades_15000ev_3.5-6.5_2.pickle", "rb"))
print('cascades DONE!')
tracks2, track_records2 = pickle.load(open(f"/dss/pone/pone_events/tracks_15000ev_3.5-6.5_2.pickle", "rb"))
print('tracks DONE!')
stracks2, strack_records2 = pickle.load(open(f"/dss/pone/pone_events/stracks_15000ev_3.5-6.5_2.pickle", "rb"))
print('starting tracks DONE!')

cascades = cascades1 + cascades2
cascade_records = cascade_records1 + cascade_records2
tracks = tracks1 + tracks2
track_records = track_records1 + track_records2
stracks = stracks1 + stracks2
strack_records = strack_records1 + strack_records2

In [ ]:
cascade_labels, track_labels, strack_labels = event_labelling(track_records, strack_records, cascade_records, det_hull=(75.0, 1000.0))

all_events = cascades + tracks + stracks
all_records = cascade_records + track_records + strack_records
all_labels = cascade_labels + track_labels + strack_labels
print(len(all_events), len(all_records), len(all_labels))

In [ ]:
#pickle.dump(data_array, open("/dss/pone/pone_events/features_arrays/data_array_90k_k8_R75.pickle", "wb"))
data_array = pickle.load(open("/dss/pone/pone_events/features_arrays/data_array_90k_k8_R75.pickle", "rb"))

In [ ]:
import random
random.seed(234567)

indices = np.arange(len(data_array))
random.shuffle(indices)


shuffled_data = [data_array[i] for i in indices]
split_test = int(len(shuffled_data)*0.9)
training_data = shuffled_data[:split_test]
split_val = int(len(training_data)*0.9)
train_dataset = training_data[:split_val]
val_dataset = training_data[split_val:]

test_dataset = shuffled_data[split_test:]
test_indices = indices[split_test:]

In [ ]:
epochs = 200
num_classes=4
patience = 200
batch_sizes = [200]
learning_rates = [0.001]
use_writer = False
print_step = 1
k_values = [8]
schedulers = ['ReduceLROnPlateau']#, 'CosineAnnealingLR', 'OneCycleLR', 'MultiStepLR']
label_map = {0:"Contained cascade", 1:'Throughgoing Track', 2:"Starts in detector", 3:"Rest of events"}
best_acc, patience_count = 0, 0
all_trains_acc, all_vals_acc, all_trains_loss, all_vals_loss = [], [], [], []
num_node_features=15

